In [4]:
import h5py
import math
import numpy as np
import os

# Inputs

In [5]:
numEpisodes = range(0,1,1)
insite_version = 3.3
raymobtimepath = './ray_tracing_data_s008_carrier60GHz'
filePrefix = 'rosslyn_mobile_60GHz_ts0.1s_V_Lidar_e'
extension ='.hdf5'
numOfInvalidChannels = 0
arrayName = 'allEpisodeData'
final_data = [] #[Episodes Scenes RxTxPairs NumofRays NumofParams]

# Reading Data

## Read array


In [6]:
def readArray(fileName, arrayName):
   openFile = h5py.File(fileName, 'r')
   #tst = list(array.keys())
   array = openFile[arrayName]
   zyx = np.arange(array.ndim)[::-1]  
   array = np.transpose(array,zyx)
   return array

In [7]:
def Rdata(fileName):
   arrayName = 'allEpisodeData'
   AllEpisodeData = readArray(fileName,arrayName)
   dimension = len(AllEpisodeData.shape)
   if dimension == 3:
      AllEpisodeData = AllEpisodeData[np.newaxis]
   return(AllEpisodeData)

In [8]:
def DiscardingInvalids(allEpisodeData,sceneNum,rxtxPair):
	channelRays=np.squeeze(allEpisodeData[:,:,rxtxPair-1,sceneNum-1])
	[numRaysPerTxRxPair, numParametersPerRay]=channelRays.shape
	theNaN = np.isnan(channelRays)
	sumOfNaN=np.count_nonzero(theNaN)
	if  (sumOfNaN > 0):  	#there is at least one NaN, so, need to check
	    	if  (sumOfNaN == numRaysPerTxRxPair*numParametersPerRay): #the whole channel is invalid (there is not a single valid ray)
	        	channelRays = -1
	    	else:
	    		if (theNaN[:,1]=='False'):
	        		validRays = 1
	        		channelRays = channelRays[validRays,:]
	return(channelRays)

In [9]:
for index in numEpisodes:
   fileName = os.path.join(raymobtimepath,filePrefix+'{}'.format(index)+extension) #filename in hdf5 extension, without the episode counter
   print ('Processing {}'.format(fileName))
   allEpisodeData = Rdata(fileName)
   [numPathParameters, maxNumPaths, numTxRxPairs, numScenes]=allEpisodeData.shape
   i_scenes =range(1,numScenes+1,1)
   i_pair=range(1,numTxRxPairs+1,1)


   for index_s in i_scenes:
        for index_p in i_pair:
            print ('Processing episode  {}'.format(index)+' scene {}'.format(index_s)+' of receiver {}'.format(index_p))
            channelRays=DiscardingInvalids(allEpisodeData,index_s,index_p)
            channel_type = type(channelRays)
            #print(channelRays)
            if channel_type == int:
                if  (channelRays == -1): #value returned when all rays are invalid
                    numOfInvalidChannels = numOfInvalidChannels + 1
                    continue #next Tx / Rx pair
            elif channel_type == np.ndarray:
                if  (channelRays.all() == -1): #value returned when all rays are invalid
                    numOfInvalidChannels = numOfInvalidChannels + 1
                    continue #next Tx / Rx pair

            [numParameters, numPaths] = channelRays.shape   #support files with 7 or 8 ray information (8 is ray phase)
            doesItIncludeRxAngle = 0

            if (numParameters >= 9):
                doesItIncludeRxAngle = 1

            gainMagnitude = channelRays[0,:] # received power in dBm
            timeOfArrival = channelRays[1,:] #time of arrival in seconds
            AoD_el = channelRays[2,:] #elevation angle of departure in degrees
            AoD_az = channelRays[3,:] #azimuth angle of departure in degrees
            AoA_el = channelRays[4,:] #elevation angle of arrival in degrees
            AoA_az = channelRays[5,:] #azimuth angle of arrival in degrees
            isLOS = channelRays[6,:]  #flag 1 for LOS ray, flag 0 for NLOS ray
            if (insite_version <= 3.2):
                gainPhase = 2*math.pi*np.random.uniform(0,1,len(gainMagnitude))
                # it was not specified in first 5gm data version, so if we don't
				# have it, generate uniformly distributed phase
            else:
                gainPhase = channelRays[7,:]*math.pi/180 #ray phases recorded from InSite
            if doesItIncludeRxAngle:
                RxAngle = channelRays[8,:]
			
            gainMagnitude = np.power(10,(0.1*gainMagnitude)) #transform to linear
            complexGain=np.power(gainMagnitude,1j * gainPhase) #consider complex gain
            isChannelLOS = sum(isLOS)
            #example: Return gainMagnitude (options - timeOfArrival, AoD_az, AoD_el, 
            # AoA_az, AoA_el, isLOS, gainPhase, RxAngle)
            # print(gainMagnitude)

Processing ./ray_tracing_data_s008_carrier60GHz\rosslyn_mobile_60GHz_ts0.1s_V_Lidar_e0.hdf5
Processing episode  0 scene 1 of receiver 1
Processing episode  0 scene 1 of receiver 2
Processing episode  0 scene 1 of receiver 3
Processing episode  0 scene 1 of receiver 4
Processing episode  0 scene 1 of receiver 5
Processing episode  0 scene 1 of receiver 6
Processing episode  0 scene 1 of receiver 7
Processing episode  0 scene 1 of receiver 8
Processing episode  0 scene 1 of receiver 9
Processing episode  0 scene 1 of receiver 10
ipykernel_launcher:10: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
